In [251]:
%matplotlib widget

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from ProcessResults import *
from DatasetTester import *
from GetLandmarks_og import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit





from scipy import linalg
import seaborn as sns


In [252]:
MEEI_path = Path('./MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

In [253]:
pat_info = pd.read_pickle('MEEI_pat_info.pkl')

with open('all_pats_processed_data.pickle','rb') as f:
    loaded_pats = pickle.load(f)

In [254]:
pat_info

,Category,Sub-category,#,Side,Gender,Age,Path,HB_scale
0,Flaccid,Complete,1,Left,Male,54,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
1,Flaccid,Complete,2,Right,Female,16,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
2,Flaccid,Complete,3,Right,Male,38,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
3,Flaccid,Complete,4,Left,Female,31,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
4,Flaccid,Complete,5,Left,Female,52,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
5,Flaccid,Mild,1,Left,Male,72,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
6,Flaccid,Mild,2,Right,Female,55,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
7,Flaccid,Mild,3,Left,Male,75,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
8,Flaccid,Moderate,1,Left,Male,51,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4
9,Flaccid,Moderate,3,Left,Female,62,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4


In [255]:
# x - choose one
all_ids = loaded_pats[0]
all_imgs = loaded_pats[1]
all_lms = loaded_pats[2]
all_features = loaded_pats[3]

# y
all_hb = loaded_pats[4]

In [256]:
# Find NaN and Inf values (this step should no longer be necessary)
bad_indices = np.where(np.isnan(all_features).sum(axis=1) > 0)[0]
bad_indices = np.append(bad_indices, np.where(np.isinf(all_features).sum(axis=1) > 0)[0])
bad_indices.sort()
bad_indices

array([], dtype=int64)

In [257]:
# drop bad indices 
# x
all_ids = np.delete(all_ids, bad_indices, axis=0)
all_imgs = np.delete(all_imgs, bad_indices, axis=0)
all_lms = np.delete(all_lms, bad_indices, axis=0)
all_features = np.delete(all_features, bad_indices, axis=0)
X = all_features

# y
all_hb = np.delete(all_hb, bad_indices, axis=0)
new_hb = np.array([1 if x >= 2 else 0 for x in all_hb])
Y = new_hb # comment this out if you want to use the original hb values (6 categories)

all_features.shape, all_hb.shape

((408, 29), (408,))

In [258]:
plot_df = pd.DataFrame(all_features, columns = ['f'+str(i) for i in range(0, 29)])

In [259]:
plot_df['hb'] = all_hb

In [260]:
def feature_plot(df, which_feature, group_by):
    sns.displot(df, x=which_feature, hue=group_by, element="step")

#for i in range(0, 29):
#    feature_plot(plot_df, 'f'+str(i), 'hb')

In [261]:
# Split IDs into train and test
random.seed(59)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 31)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

Training IDs: [ 0  1  2  5  6  7  8  9 10 11 14 18 19 20 21 24 27 28 29 31 33 34 35 38
 39 40 41 42 43 44 46]
Testing IDs: [ 3  4 12 13 15 16 17 22 23 25 26 30 32 36 37 45 47 48 49 50]


In [262]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train, x_test = X[train_idx], X[test_idx]
y_train, y_test = Y[train_idx], Y[test_idx]

print('Training x shape:', x_train.shape)
print('Training y shape:', y_train.shape)
print('Testing x shape:', x_test.shape)
print('Testing y shape:', y_test.shape)

Training x shape: (248, 29)
Training y shape: (248,)
Testing x shape: (160, 29)
Testing y shape: (160,)


In [263]:
train_indx, test_indx = next(
    GroupShuffleSplit(random_state=7, test_size=0.3).split(X, Y, all_ids))

X_train, X_test, Y_train, Y_test = \
    X[train_indx], X[test_indx], Y[train_indx], Y[test_indx]
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((280, 29), (128, 29), (280,), (128,))

In [264]:
x_train = X_train
x_test = X_test
y_train = Y_train
y_test = Y_test

In [265]:
np.unique(all_ids[train_indx]), np.unique(all_ids[test_indx])

(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 14, 16, 19, 21, 23, 24,
        25, 26, 27, 28, 31, 32, 34, 36, 39, 40, 41, 42, 43, 45, 46, 47, 48,
        49]),
 array([ 1, 10, 13, 15, 17, 18, 20, 22, 29, 30, 33, 35, 37, 38, 44, 50]))

In [266]:
x_train[0], y_train[0]

(array([  1.645194  ,   2.8099802 ,  12.804038  ,   1.0514685 ,
          0.02872195,   0.04908276,   0.22726855,   2.5692458 ,
          1.0851028 ,   1.0569661 ,   1.3587397 ,   0.7349562 ,
          0.68135345,   0.78855604,  14.76165   ,   1.2982798 ,
          1.564689  ,   1.6300427 ,   0.1822432 ,   0.43270245,
          0.4529679 ,   1.7084181 ,  11.120797  , -18.251646  ,
          1.0966494 ,   0.7621016 ,   0.7131144 ,   0.29012558,
          0.12468714], dtype=float32),
 1)

In [267]:
x_test[0], y_test[0]

(array([ 2.1688318 ,  5.7567234 ,  7.4379115 ,  1.0764056 ,  0.03787135,
         0.10081324,  0.13055022,  0.7424527 ,  1.0363585 ,  1.0367308 ,
         1.0354205 ,  0.10949325,  0.09149812,  0.12822627,  2.7387393 ,
         1.0146147 ,  1.5828909 ,  1.4162164 ,  0.16872515,  0.3655438 ,
         0.351555  ,  1.3814355 ,  0.09058161, -0.2640291 ,  1.0116374 ,
         0.78680843,  0.7529882 ,  0.268227  ,  0.11025678], dtype=float32),
 1)

In [268]:
y_train[220:301]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [269]:
#le = LabelEncoder()
#y_train = le.fit_transform(y_train)

In [270]:
y_train[220:301]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [271]:
clf = MLPClassifier(solver='adam', hidden_layer_sizes=(8,7), random_state=1,
                                    max_iter=10000, activation='relu', learning_rate='constant',
                                    learning_rate_init=0.2045,momentum=0.19)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [272]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %2f" % (rmse))
print('Accuracy:', round(accuracy_score(y_test, y_pred), 3))

RMSE: 0.433013
Accuracy: 0.812


In [273]:
y_pred[20:30], y_test[20:30]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

### incorrect predictions

In [274]:
test_idx[np.where(y_pred != y_test)[0]]

array([136, 137, 138, 139, 140, 141, 142, 143, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191])

In [275]:
test_idx[np.where(y_pred != y_test)[0]]/8

array([17.   , 17.125, 17.25 , 17.375, 17.5  , 17.625, 17.75 , 17.875,
       22.   , 22.125, 22.25 , 22.375, 22.5  , 22.625, 22.75 , 22.875,
       23.   , 23.125, 23.25 , 23.375, 23.5  , 23.625, 23.75 , 23.875])

### false positives: said ill, actually healthy

In [276]:
test_idx[np.where(y_pred > y_test)[0]]

array([136, 137, 138, 139, 140, 141, 142, 143, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191])

In [277]:
test_idx[np.where(y_pred > y_test)[0]]/8

array([17.   , 17.125, 17.25 , 17.375, 17.5  , 17.625, 17.75 , 17.875,
       22.   , 22.125, 22.25 , 22.375, 22.5  , 22.625, 22.75 , 22.875,
       23.   , 23.125, 23.25 , 23.375, 23.5  , 23.625, 23.75 , 23.875])

### false negatives: said healthy, actually ill

In [278]:
test_idx[np.where(y_pred < y_test)[0]]

array([], dtype=int64)

In [279]:
test_idx[np.where(y_pred < y_test)[0]]/8

array([], dtype=float64)

### Now try data standardaization / normalization

In [280]:
# Before transformation
x_train[0]

array([  1.645194  ,   2.8099802 ,  12.804038  ,   1.0514685 ,
         0.02872195,   0.04908276,   0.22726855,   2.5692458 ,
         1.0851028 ,   1.0569661 ,   1.3587397 ,   0.7349562 ,
         0.68135345,   0.78855604,  14.76165   ,   1.2982798 ,
         1.564689  ,   1.6300427 ,   0.1822432 ,   0.43270245,
         0.4529679 ,   1.7084181 ,  11.120797  , -18.251646  ,
         1.0966494 ,   0.7621016 ,   0.7131144 ,   0.29012558,
         0.12468714], dtype=float32)

#### We try a few different scalers and fit them (only) to the training data

 - StandardScaler
 - MinMaxScaler
 - PowerTransformer (Box-Cox)
 - QuantileTransformer (Gaussian)

See https://scikit-learn.org/stable/modules/preprocessing.html#non-linear-transformation


In [281]:
scaler = StandardScaler().fit(x_train) # standardize features by removing the mean and scaling to unit variance
#scaler = MinMaxScaler().fit(x_train) # scale features to lie between 0 and 1
#scaler = PowerTransformer(method='box-cox').fit(x_train) # apply a power transform featurewise to make data more Gaussian-like
#scaler = QuantileTransformer(output_distribution='normal').fit(x_train) # transform features using quantiles information

x_train_std, x_test_std = scaler.transform(x_train), scaler.transform(x_test)
x_train, x_test = x_train_std, x_test_std

In [282]:
# After transformation
x_train[0]

array([ 7.5686336e-01,  6.6375923e-01,  2.2879686e+00,  8.4072119e-01,
        7.5584733e-01,  6.6106766e-01,  2.2951081e+00,  2.0860639e+00,
        1.6358169e+00, -5.7756376e-01,  5.2196631e+00,  1.0356594e+00,
        8.8419801e-01,  1.0915219e+00,  4.1579514e+00,  4.7211723e+00,
        1.9037715e-01,  3.5391693e-04, -6.8530107e-01,  4.7916758e-01,
        5.4603714e-01,  8.5167414e-01,  3.7869074e+00, -3.7612903e+00,
        1.3448803e+00, -5.4771471e-01, -7.6003677e-01,  2.8627628e-01,
        3.7579951e-01], dtype=float32)

In [283]:
clf = MLPClassifier(solver='adam', hidden_layer_sizes=(56,), random_state=1,
                                    max_iter=10000, activation='relu', learning_rate='constant',
                                    learning_rate_init=0.2045,momentum=0.19)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [284]:
#from tpot import TPOTClassifier
#tpot_clf = TPOTClassifier(generations=5, population_size=100, 
#                          verbosity=2, offspring_size=100, scoring='accuracy', cv=6)
#Training and prediction
#tpot_clf.fit(x_train,y_train)
#y_pred = tpot_clf.predict(x_test)

In [285]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %2f" %(rmse))
print('Accuracy:', round(accuracy_score(y_test, y_pred), 3))

RMSE: 0.459279
Accuracy: 0.789


# Cross-validation

In [286]:
from sklearn.model_selection import cross_val_score
#clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, Y, 
                         cv=GroupShuffleSplit(n_splits=20, random_state=7, test_size=0.3).split(X, Y, all_ids))
scores

array([0.8046875, 0.7265625, 0.8125   , 0.796875 , 0.6875   , 0.734375 ,
       0.8203125, 0.8046875, 0.703125 , 0.734375 , 0.6875   , 0.8671875,
       0.7734375, 0.875    , 0.7890625, 0.828125 , 0.7734375, 0.875    ,
       0.78125  , 0.6875   ])

In [287]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.78 accuracy with a standard deviation of 0.06


In [288]:
##TODO: make a graph / table on CV results for different type of scaling 

In [289]:
from tensorflow import keras

In [297]:
# initialize model
model = keras.models.Sequential()

# add input layer
model.add(keras.layers.Dense(
    units=50,
    input_dim=x_train.shape[1],
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh') 
)
# add hidden layer
model.add(
    keras.layers.Dense(
        units=50,
        input_dim=(8,7),
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh')
    )
# add output layer
model.add(
    keras.layers.Dense(
        units=1,
        input_dim=50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax')
    )

# define SGD optimizer
sgd_optimizer = keras.optimizers.SGD(
    #lr=0.002045, momentum=0.1909
    lr=0.001, decay=1e-7, momentum=0.9
)
# compile model
model.compile(
    optimizer=sgd_optimizer,
    loss='categorical_crossentropy'
)

/opt/anaconda/envs/bp_tf/lib/python3.10/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [298]:
# train model
history = model.fit(
    x_train, y_train,
    batch_size=64, epochs=100,
    verbose=1, validation_split=0.2
)

Epoch 1/100
4/4 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/100
4/4 [==============================] - 0s

In [299]:
# calculate training accuracy
y_train_pred = model.predict(x_train, verbose=0)
correct_preds = np.sum(np.where(y_train == y_train_pred))
train_acc = correct_preds / y_train.shape[0]

#print(f'Training accuracy: {(train_acc * 100):.2f}')

# calculate testing accuracy
y_test_pred = model.predict(x_test, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]

#print(f'Test accuracy: {(test_acc * 100):.2f}')

In [300]:
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("RMSE: %2f" %(rmse))
print('Accuracy:', round(accuracy_score(y_test, y_test_pred), 3))

RMSE: 0.433013
Accuracy: 0.812
